<h1 style="color: #2e6c80; text-align: center;"><span style="text-decoration: underline;">Trabajo Final de CIS311</span></h1>
<h2 style="color: #2e6c80;">&nbsp;</h2>
<h4 style="color: #2e6c80;"><span style="color: #000000;">Alumno: Brian Alexander Jaramillo Ramirez</span></h4>
<h4 style="color: #2e6c80;"><span style="color: #000000;">C&oacute;digo: 20131667</span></h4>
<h4 style="color: #2e6c80;"><span style="color: #000000;">Profesora: Carla Sol&iacute;s</span></h4>
<h2 style="color: #2e6c80;">1. Introducci&oacute;n</h2>
<p style="text-align: justify;">El presente trabajo se enfoca en el impacto de la pandemia del COVID-19 por todo el mundo. El brote inicial comenz&oacute; a fines del a&ntilde;o 2019 en la ciudad de Wuhan en China. Y en poco tiempo se expandi&oacute; a otros pa&iacute;ses as&iacute;aticos, hasta extenderse a Europa, &Aacute;frica y Am&eacute;rica. El alto contagio que lo caracteriza provoc&oacute; que a nivel global se tomasen medidas restrictivas para reducir el n&uacute;mero de infectados. Sin embargo, tanto los enfermos como los fallecidos producto del virus se contaron por cientos de miles, ya que las medidas no funcionaron, y reci&eacute;n con la vacunaci&oacute;n han comenzado a reducirse los casos. A continuaci&oacute;n, se realizar&aacute; un an&aacute;lisis estad&iacute;stico tanto de los infectados, muertos y vacunados entre los principales pa&iacute;ses.</p>

<h2 style="color: #2e6c80;">2. Base de datos</h2>
<h3 style="color: #2e6c80;">2.1 Importando bibliotecas</h3>

Primero, se importarán las bibliotecas necesarias que se utilizarán a lo largo de la creación de la base de datos y los gráficos a implementar. Por un lado, `pandas` permite manejar bases de datos, mientras que `numpy` ayuda a realizar cálculos numéricos y de datos. Por otro lado, con `altair` se implementan diversos gráficos. Finalmente, se usará `wget` para descargar las bases de datos necesarias.  

In [5]:
import pandas as pd
import numpy as np
import altair as alt
import wget

<h3 style="color: #2e6c80;">2.2 Descargando la base de datos</h3>

La base de datos es de una fuente confiable ya que ha sido descargada de la página web "Coronavirus Source Data". Estos datos se basan en los recopilados a nivel mundial por parte de la Universidad Johns Hopkins. Cabe señalar que los datos cubren el periodo de enero de 2020 hasta el 24 de junio de 2021 (esto es debido a la fecha en que se realizó este trabajo). A continuación se adjunta el link: 

<p><a title="Our World In Data" href="https://ourworldindata.org/" target="_blank">https://ourworldindata.org/</a></p>

Definimos la ruta de descarga en la variable "urls", para luego utilizar la librería `wget` para descargar la base que está en formato csv. Finalmente, se nos mostrará una barra con el 100% del proceso de descarga, y tendremos nuestra base descargada. Esta se llama 'owid-covid-data-csv'.

In [2]:
urls = ['https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv']
[wget.download(url) for url in urls]

100% [########################################################################]           24M / 24M

['owid-covid-data.csv']

<h3 style="color: #2e6c80;">2.3 Limpiando la base de datos</h3>

* Primero, importaremos nuestras bases de datos y les daremos un nombre para diferenciarlas.

In [92]:
base_origen = pd.read_csv('owid-covid-data.csv')

* Con el comando `shape` obtenemos el número de filas y columnas de la base de datos. Observamos que tenemos 60 columnas que corresponden a las distintas variables consideradas en la base de datos. Sin embargo, solo nos quedaremos con las variables más relevantes para el análisis estadístico. Estas serán el continente, país, fecha, casos totales de infectados, nuevos casos de infectados, total de muertes, y nuevas muertes.

In [93]:
base_origen.shape

(98011, 60)

In [94]:
base_origen = base_origen.loc[:,['continent', 'location', 'date', 'total_cases', 
                                 'new_cases', 'total_deaths', 'new_deaths']]

* Un paso importante que se debe de realizar es comprobar que no hayan datos missing (NaN). Para ello, se emplea el comando `df.isna().sum()`.

In [95]:
base_origen.isna().sum()

continent        4606
location            0
date                0
total_cases      3437
new_cases        3440
total_deaths    13520
new_deaths      13364
dtype: int64

* Observamos que hay muchos datos missing en las columnas, por lo cual, los rellenaremos con 0. Más que nada, este problema ocurre porque el impacto del virus no sucedió al mismo tiempo en todos los países. Así que según las fechas, hay países en que el virus infectó mucho después respecto al inicio de la pandemia.

In [96]:
#base_origen['continent'] = base_origen['total_cases'].fillna('No Registrado')
base_origen['total_cases'] = base_origen['total_cases'].fillna(0)
base_origen['new_cases'] = base_origen['new_cases'].fillna(0)
base_origen['total_deaths'] = base_origen['total_deaths'].fillna(0)
base_origen['new_deaths'] = base_origen['new_deaths'].fillna(0)

* Ahora, comprobaremos qué tipo de datos son nuestras variables para no tener inconvenientes más adelante al realizar los gráficos.

In [97]:
base_origen.dtypes

continent        object
location         object
date             object
total_cases     float64
new_cases       float64
total_deaths    float64
new_deaths      float64
dtype: object

* Tenemos un problema con `date`, ya que es tipo object, lo cual es incorrecto dado que representa a una fecha. Lo solucionamos transformando su formato a fecha.

In [98]:
base_origen['date'] = pd.to_datetime(base_origen['date'])
base_origen.dtypes

continent               object
location                object
date            datetime64[ns]
total_cases            float64
new_cases              float64
total_deaths           float64
new_deaths             float64
dtype: object

In [99]:
base_origen

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths
0,Asia,Afghanistan,2020-02-24,1.0,1.0,0.0,0.0
1,Asia,Afghanistan,2020-02-25,1.0,0.0,0.0,0.0
2,Asia,Afghanistan,2020-02-26,1.0,0.0,0.0,0.0
3,Asia,Afghanistan,2020-02-27,1.0,0.0,0.0,0.0
4,Asia,Afghanistan,2020-02-28,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
98006,Africa,Zimbabwe,2021-06-20,41779.0,151.0,1672.0,6.0
98007,Africa,Zimbabwe,2021-06-21,42195.0,416.0,1685.0,13.0
98008,Africa,Zimbabwe,2021-06-22,42714.0,519.0,1691.0,6.0
98009,Africa,Zimbabwe,2021-06-23,43480.0,766.0,1692.0,1.0


In [91]:
vacunacion = pd.read_csv('covid-vaccination.csv')

In [100]:
vacunacion = vacunacion.loc[:,['continent', 'location', 'date', 'total_vaccinations', 
                                 'people_vaccinated', 'people_fully_vaccinated']]

In [101]:
vacunacion

,continent,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,Asia,Afghanistan,2020-02-24,NaN,NaN,NaN
1,Asia,Afghanistan,2020-02-25,NaN,NaN,NaN
2,Asia,Afghanistan,2020-02-26,NaN,NaN,NaN
3,Asia,Afghanistan,2020-02-27,NaN,NaN,NaN
4,Asia,Afghanistan,2020-02-28,NaN,NaN,NaN
...,...,...,...,...,...,...
98206,Africa,Zimbabwe,2021-06-21,1140852.0,704001.0,436851.0
98207,Africa,Zimbabwe,2021-06-22,1146378.0,706158.0,440220.0
98208,Africa,Zimbabwe,2021-06-23,1166849.0,715056.0,451793.0
98209,Africa,Zimbabwe,2021-06-24,NaN,NaN,NaN


In [115]:
df = vacunacion[vacunacion['date']=='2021-06-24']

In [121]:
df

,continent,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated
486,Asia,Afghanistan,2021-06-24,NaN,NaN,NaN
985,NaN,Africa,2021-06-24,48436411.00,34245221.00,14541298.00
1472,Europe,Albania,2021-06-24,922886.00,537018.00,385868.00
1959,Africa,Algeria,2021-06-24,NaN,NaN,NaN
2440,Europe,Andorra,2021-06-24,NaN,NaN,NaN
...,...,...,...,...,...,...
96227,Asia,Vietnam,2021-06-24,2920248.00,2777127.00,143121.00
96839,NaN,World,2021-06-24,2838527301.00,1758545333.00,805726662.00
97281,Asia,Yemen,2021-06-24,NaN,NaN,NaN
97746,Africa,Zambia,2021-06-24,149002.00,142089.00,6913.00


In [122]:
df.groupby(by = ['continent']).sum()

,total_vaccinations,people_vaccinated,people_fully_vaccinated
continent,,,
Africa,31370075.00,18358316.00,11430192.00
Asia,1647296126.00,380180929.00,115730003.00
Europe,228983411.00,147854328.00,95618893.00
North America,357671095.00,205709521.00,160858050.00
Oceania,6999198.00,5886916.00,1112282.00
South America,112023400.00,83465191.00,28558209.00


In [119]:
pd.options.display.float_format = '{:.2f}'.format